# MEDIA BILLING RECONCILIATION WITH AGENTIC AI​

---

## DESCRIPTION

This notebook implements a **File Reader Agent** that can ingest media invoices and billing data in multiple formats and extract structured information for processing.

The agent uses CrewAI to intelligently read and parse files, making it easy to integrate with billing reconciliation workflows.

---

## 🎯 CAPABILITIES

**File Format Support:**
- 📄 **PDF** - Extract text from invoice PDFs
- 📊 **CSV** - Parse tabular billing data
- 📈 **Excel** (.xlsx, .xls) - Read spreadsheet data
- 🖼️ **Images** - OCR text extraction from scanned documents
- 📝 **Text** - Plain text file reading
- 🔖 **XML** - Parse XML structured data

**Agent Features:**
- Single unified agent for all file types
- Automatic file type detection
- Structured data extraction
- Error handling and validation
- Ready for CrewAI integration

---

## 🤖 WHAT WE'RE BUILDING

A **File Reader Agent** that:
1. Accepts file paths as input
2. Automatically detects file format
3. Extracts and structures the data
4. Returns clean, parsed content
5. Integrates with CrewAI workflows

---

## 📦 USE CASE

This agent serves as the **data ingestion layer** for the media billing reconciliation system:
- **Input**: Invoice files (PDF, CSV, Excel, images, XML)
- **Process**: Read, parse, and structure data
- **Output**: Standardized data for reconciliation agents

In [114]:
# ============================================
# INSTALL REQUIRED PACKAGES
# ============================================
# Uncomment and run to install all dependencies

# !pip install pdfplumber pandas openpyxl pillow pytesseract lxml
# !pip install crewai python-dotenv

## 📦 Step 1: Import Required Libraries

Import all necessary libraries for file reading and CrewAI integration.

In [115]:
# ============================================
# IMPORT ALL REQUIRED LIBRARIES
# ============================================
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# File reading libraries
import pdfplumber
from PIL import Image
import pytesseract
import xml.etree.ElementTree as ET

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool

print("✅ All libraries imported successfully!")
print("   • File readers: pdfplumber, pandas, PIL, pytesseract, xml")
print("   • CrewAI: Agent, Task, Crew, LLM")

✅ All libraries imported successfully!
   • File readers: pdfplumber, pandas, PIL, pytesseract, xml
   • CrewAI: Agent, Task, Crew, LLM


In [116]:
# ============================================
# FILE READER FUNCTIONS
# ============================================

def read_pdf_content(pdf_path: str) -> str:
    """Read and extract text from PDF files."""
    try:
        full_text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                text = page.extract_text()
                if text:
                    full_text += f"\n--- Page {page_num} ---\n{text}\n"
        return full_text.strip()
    except Exception as e:
        return f"Error reading PDF: {str(e)}"


def read_csv_content(csv_path: str) -> str:
    """Read and parse CSV files."""
    try:
        df = pd.read_csv(csv_path)
        return df.to_string()
    except Exception as e:
        return f"Error reading CSV: {str(e)}"


def read_excel_content(excel_path: str, sheet_name=0) -> str:
    """Read and parse Excel files."""
    try:
        df = pd.read_excel(excel_path, sheet_name=sheet_name)
        return df.to_string()
    except Exception as e:
        return f"Error reading Excel: {str(e)}"


def read_image_content(image_path: str) -> str:
    """Extract text from images using OCR."""
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text.strip()
    except Exception as e:
        return f"Error reading image: {str(e)}"


def read_xml_content(xml_path: str) -> str:
    """Parse and read XML files."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        return ET.tostring(root, encoding='unicode')
    except Exception as e:
        return f"Error reading XML: {str(e)}"


def read_text_content(text_path: str) -> str:
    """Read plain text files."""
    try:
        with open(text_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"Error reading text file: {str(e)}"

print("✅ Individual file reader functions created!")
print("   • read_pdf_content()")
print("   • read_csv_content()")
print("   • read_excel_content()")
print("   • read_image_content()")
print("   • read_xml_content()")
print("   • read_text_content()")

✅ Individual file reader functions created!
   • read_pdf_content()
   • read_csv_content()
   • read_excel_content()
   • read_image_content()
   • read_xml_content()
   • read_text_content()


In [117]:
# ============================================
# UNIVERSAL FILE READER TOOL (CrewAI Tool)
# ============================================

@tool("read_file_tool")
def read_file_tool(file_path: str) -> str:
    """
    Universal file reader tool that automatically detects and reads various file formats.
    Supports: PDF, CSV, Excel, Images (OCR), XML, and Text files.
    
    Args:
        file_path: Path to the file to read
        
    Returns:
        Extracted content as string
    """
    file_path = Path(file_path)
    
    if not file_path.exists():
        return f"❌ Error: File not found at {file_path}"
    
    suffix = file_path.suffix.lower()
    
    print(f"📂 Reading file: {file_path.name} (type: {suffix})")
    
    # Route to appropriate reader based on file extension
    if suffix == '.pdf':
        content = read_pdf_content(str(file_path))
    elif suffix == '.csv':
        content = read_csv_content(str(file_path))
    elif suffix in ['.xlsx', '.xls']:
        content = read_excel_content(str(file_path))
    elif suffix in ['.png', '.jpg', '.jpeg', '.bmp', '.tiff']:
        content = read_image_content(str(file_path))
    elif suffix == '.xml':
        content = read_xml_content(str(file_path))
    elif suffix == '.txt':
        content = read_text_content(str(file_path))
    else:
        content = f"⚠️ Unsupported file type: {suffix}"
    
    print(f"✅ Successfully read {len(content)} characters")
    return content

print("✅ CrewAI File Reader Tool created!")
print("   Tool name: 'read_file_tool'")
print("   Supports: PDF, CSV, Excel, Images, XML, Text")

✅ CrewAI File Reader Tool created!
   Tool name: 'read_file_tool'
   Supports: PDF, CSV, Excel, Images, XML, Text


In [118]:
# ============================================
# LOAD ENVIRONMENT AND INITIALIZE LLM
# ============================================

load_dotenv()

# Get OpenAI API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️  Warning: OPENAI_API_KEY not found in environment")
    print("   Please add it to your .env file")
else:
    print("✅ OpenAI API key loaded")

# Initialize LLM
llm = LLM(
    model="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0.7
)

print("✅ LLM initialized: gpt-4o")

✅ OpenAI API key loaded
✅ LLM initialized: gpt-4o


In [119]:
# ============================================
# CREATE FILE READER AGENT
# ============================================

file_reader_agent = Agent(
    role='File Data Extraction Specialist',
    goal='Read and extract data from various file formats including PDF, CSV, Excel, images, XML, and text files',
    backstory="""You are an expert in data extraction and file parsing. 
    You can read any type of document - from PDFs and spreadsheets to scanned images 
    and XML files. You extract information accurately and structure it in a clear, 
    usable format. You handle invoices, billing documents, and financial records 
    with precision.""",
    llm=llm,
    tools=[read_file_tool],
    verbose=True,
    allow_delegation=False
)

print("✅ File Reader Agent created!")
print(f"   Role: {file_reader_agent.role}")
print(f"   Tools: {[tool.name for tool in file_reader_agent.tools]}")

✅ File Reader Agent created!
   Role: File Data Extraction Specialist
   Tools: ['read_file_tool']


In [120]:
# ============================================
# CREATE FILE READING TASK
# ============================================

def create_file_reading_task(file_path: str, extraction_focus: str = "all data") -> Task:
    """
    Create a task for reading and extracting data from a file.
    
    Args:
        file_path: Path to the file to read
        extraction_focus: What specific data to focus on (default: "all data")
    
    Returns:
        Task object configured for file reading
    """
    task = Task(
        description=f"""Read and extract data from the following file:
        
File Path: {file_path}
Extraction Focus: {extraction_focus}

Your tasks:
1. Use the read_file_tool to read the file
2. Extract and structure the relevant information
3. If it's an invoice or billing document, identify:
   - Vendor/supplier name
   - Invoice number and date
   - Line items with descriptions and amounts
   - Total amount
   - Any other relevant billing information
4. Present the extracted data in a clear, structured format

Provide a comprehensive summary of the file contents.""",
        agent=file_reader_agent,
        expected_output="""A structured summary of the file contents including:
        - File type and basic metadata
        - Key data points extracted
        - For invoices: vendor, invoice number, date, line items, total
        - Any notable information or patterns found"""
    )
    
    return task

print("✅ File reading task creator function defined!")
print("   Usage: task = create_file_reading_task('path/to/file.pdf')")

✅ File reading task creator function defined!
   Usage: task = create_file_reading_task('path/to/file.pdf')


## 🚀 Step 6: Create and Run Crew

Assemble the crew and execute the file reading task.

In [121]:
# ============================================
# CREATE AND RUN FILE READER CREW
# ============================================

def process_file(file_path: str, extraction_focus: str = "all data"):
    """
    Process a file using the File Reader Agent and Crew.
    
    Args:
        file_path: Path to the file to process
        extraction_focus: Specific data to focus on
        
    Returns:
        Extracted data and analysis
    """
    print(f"\n{'='*70}")
    print(f"🚀 Processing File: {Path(file_path).name}")
    print(f"{'='*70}\n")
    
    # Create task for this file
    task = create_file_reading_task(file_path, extraction_focus)
    
    # Create crew
    crew = Crew(
        agents=[file_reader_agent],
        tasks=[task],
        process=Process.sequential,
        verbose=True
    )
    
    # Execute
    result = crew.kickoff()
    
    print(f"\n{'='*70}")
    print(f"✅ File Processing Complete!")
    print(f"{'='*70}\n")
    
    return result

print("✅ File processing function created!")
print("   Usage: result = process_file('path/to/invoice.pdf')")

✅ File processing function created!
   Usage: result = process_file('path/to/invoice.pdf')
